Below are live tests that will run using your actual Google Sheets credentials and interact with Google's APIs. If there are any failures, test workbooks may be left behind.

In [1]:
import datetime as dt
import random

import oauth2client
import pandas as pd

import datasheets

# Client

In [2]:
# Can create a non-service account
nonsvc_client = datasheets.Client()
assert isinstance(nonsvc_client.credentials, oauth2client.client.OAuth2Credentials)
assert not hasattr(nonsvc_client.credentials, 'service_account_email')
del nonsvc_client

In [3]:
# Can create a service account
service_client = datasheets.Client(service=True)
assert isinstance(service_client.credentials, oauth2client.service_account.ServiceAccountCredentials)
assert hasattr(service_client.credentials, 'service_account_email')
del service_client

In [4]:
filename = 'datasheets_manual_testing_client_' + str(random.randint(0, 100000))
client = datasheets.Client()

# Can create a workbook
wkb = client.create_workbook(filename)
assert isinstance(wkb, datasheets.Workbook)

# Can fetch a workbook
wkb_again = client.fetch_workbook(filename)
assert isinstance(wkb_again, datasheets.Workbook)
assert wkb.file_id == wkb_again.file_id

# Can fetch workbooks info
wkb_info = client.fetch_workbooks_info()
assert isinstance(wkb_info, pd.DataFrame)
assert len(wkb_info) > 0  # We know at least this workbook exists
del wkb_info

# Can delete a workbook and assert it no longer exists
client.delete_workbook(filename)
wkb_info = client.fetch_workbooks_info()
assert len(wkb_info[wkb_info['name'] == filename]) == 0

del client, wkb, wkb_again, wkb_info

# Workbook

In [5]:
filename = 'datasheets_manual_testing_workbook_' + str(random.randint(0, 100000))
tabname = 'mytab'
client = datasheets.Client()


wkb = client.create_workbook(filename)

# Can create a tab
tab = wkb.create_tab(tabname)
assert isinstance(tab, datasheets.Tab)

# Can fetch a tab
tab_again = wkb.fetch_tab(tabname)
assert isinstance(tab_again, datasheets.Tab)
assert tab.tab_id == tab_again.tab_id

# Can fetch all tab names
tab_names = wkb.fetch_tab_names()
assert isinstance(tab_names, pd.DataFrame)
assert len(tab_names) == 2

# Can fetch permissions
perms = wkb.fetch_permissions()
assert isinstance(perms, pd.DataFrame)
assert len(perms) > 0  # We know at least we have permissions

# Can delete a tab
wkb.delete_tab(tabname)

client.delete_workbook(filename)
del client, wkb, tab, tab_again, tab_names

# Tab

In [6]:
df = pd.DataFrame(
    data=[
        ['a', 1.23, 'this is a sentence.'],
        [3.23, 7.0, dt.date(2016, 1, 1), None],
        [],
        [dt.datetime(2010, 8, 7, 16, 13), True, 0.19, 0.54, None],
        [None, dt.time(10, 3), 3.23, None, None],
        [None, None, None, None, None]
    ], columns=('alpha', 'beta', 'gamma', 'delta', 'epsilon')
)
df

,alpha,beta,gamma,delta,epsilon
0,a,1.23,this is a sentence.,NaN,None
1,3.23,7,2016-01-01,NaN,None
2,None,None,None,NaN,None
3,2010-08-07 16:13:00,True,0.19,0.54,None
4,None,10:03:00,3.23,NaN,None
5,None,None,None,NaN,None


In [7]:
filename = 'datasheets_manual_testing_tab_' + str(random.randint(0, 100000))
tabname = 'mytab'

client = datasheets.Client()
wkb = client.create_workbook(filename)
tab = wkb.create_tab(tabname)

# Can insert data
tab.insert_data(df)

# Can fetch back the data we inserted
df_fetched = tab.fetch_data()
assert df_fetched.drop('index0', axis=1).equals(df)

# Can append data
tab.append_data(df)
df_fetched_again = tab.fetch_data()
assert len(df_fetched_again) == (2 * len(df_fetched))

# Can clear data
tab.clear_data()
df_after_clearing = tab.fetch_data()
assert df_after_clearing.empty

client.delete_workbook(filename)

del client, wkb, tab, df_fetched, df_fetched_again, df_after_clearing

# Convenience

In [8]:
filename = 'datasheets_manual_testing_tab_' + str(random.randint(0, 100000))
tabname = 'mytab'


# Can create a tab in a new workbook
tab = datasheets.create_tab_in_new_workbook(filename, tabname)
assert isinstance(tab, datasheets.Tab)
assert tab.workbook.filename == filename

# Delete this workbook
client = datasheets.Client()
client.delete_workbook(filename)

In [9]:
filename = 'datasheets_manual_testing_tab_' + str(random.randint(0, 100000))
tabname = 'mytab'

client = datasheets.Client()
client.create_workbook(filename)

# Can create a tab in an existing workbook
tab = datasheets.create_tab_in_existing_workbook(filename, tabname)
assert isinstance(tab, datasheets.Tab)
assert tab.workbook.filename == filename

# Delete this workbook
client.delete_workbook(filename)

# Remove Left-Over Files

In [10]:
client = datasheets.Client()

wkb_info = client.fetch_workbooks_info()
mask = wkb_info.name.str.contains('datasheets_manual_testing')
workbook_names = wkb_info.loc[mask]

for file_id in workbook_names['id']:
    print("Deleting workbook with file_id='{}'".format(file_id))
    client.delete_workbook(file_id=file_id)